In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize

# === NumPy, SciPy e otimização ===
import numpy as np
from scipy.optimize import minimize
  
# === Visualização ===
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram, array_to_latex, plot_state_city
from rustworkx.visualization import mpl_draw as draw_graph
import time

# === Estruturas de grafos ===
import networkx as nx
import rustworkx as rx

# === Qiskit: circuitos, operadores e transpiler ===
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit.circuit.library import QAOAAnsatz, PauliEvolutionGate
from qiskit.synthesis.evolution import LieTrotter
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

# === Qiskit Aer (simulação local) ===
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator as EstimatorV2, Sampler as AerSampler

# === Qiskit Runtime (execução em hardware IBM) ===
from qiskit_ibm_runtime import QiskitRuntimeService, Session, EstimatorV2, SamplerV2

# === Qiskit Primitives ===
# from qiskit.primitives import Sampler, BaseSamplerV2

# === Qiskit Optimization === 
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.minimum_eigensolvers import QAOA
from qiskit_optimization.optimizers import COBYLA, NELDER_MEAD

# ---- Importações necessárias (atualizadas para Qiskit 2.x) ----
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_aer import AerSimulator
from qiskit.primitives import StatevectorEstimator as Estimator  # Correção: Use StatevectorEstimator
from qiskit.primitives import StatevectorSampler as Sampler  # Correção: Use StatevectorSampler
from qiskit.circuit.library import QAOAAnsatz  # Novo: QAOAAnsatz em vez de QAOA
from qiskit.quantum_info import SparsePauliOp  # Para operadores
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager  # transpiler para versões antigas/compatíveis
from qiskit import transpile
import numpy as np
from scipy.optimize import minimize  # Otimizador clássico externo 

In [2]:
# Dados do problema (20 itens)
valor_itens = [220, 208, 198, 192, 180, 180, 165, 162, 160, 158, 155, 130, 125, 122, 120, 118, 115, 110, 105, 101]
peso_itens = [80, 82, 85, 70, 72, 70, 66, 50, 55, 25, 50, 55, 40, 48, 59, 32, 22, 60, 30, 32]
capacidade = 500
qtd_itens = len(valor_itens)
penalidade = 500

print(f"Capacidade: {capacidade}")
print(f"Peso dos itens: {peso_itens}, Quantidade de itens: {qtd_itens}")
print(f"Valor dos itens: {valor_itens}, Quantidade de itens: {qtd_itens}")

Capacidade: 500
Peso dos itens: [80, 82, 85, 70, 72, 70, 66, 50, 55, 25, 50, 55, 40, 48, 59, 32, 22, 60, 30, 32], Quantidade de itens: 20
Valor dos itens: [220, 208, 198, 192, 180, 180, 165, 162, 160, 158, 155, 130, 125, 122, 120, 118, 115, 110, 105, 101], Quantidade de itens: 20


In [3]:
def cria_qubo(valor_itens, peso_itens, capacidade, penalidade): 
    n = len(valor_itens)
    qubo_matrix = np.zeros([n, n])
    for i in range(n):
        qubo_matrix[i, i] = -valor_itens[i] - penalidade * (peso_itens[i]**2)
        for j in range(i + 1, n):
            qubo_matrix[i, j] = 2 * penalidade * peso_itens[i] * peso_itens[j]
    for i in range(n):
        qubo_matrix[i, i] += 2 * penalidade * peso_itens[i] * capacidade
    return qubo_matrix

qubo_matrix = cria_qubo(valor_itens, peso_itens, capacidade, penalidade)

In [4]:
def hamiltanino_de_custo(qubo_matrix):
    n = len(qubo_matrix)
    h_custo = SparsePauliOp("I" * n, coeffs=[0])
    for i in range(n):
        pauli_str_diag = ["I"] * n
        pauli_str_diag[i] = "Z"
        coef_diag = -qubo_matrix[i, i] / 2
        h_custo += SparsePauliOp("".join(pauli_str_diag), coeffs=[coef_diag])
    for i in range(n):
        for j in range(i + 1, n):
            if qubo_matrix[i, j] != 0:
                pauli_str_quad = ["I"] * n
                pauli_str_quad[i] = "Z"
                pauli_str_quad[j] = "Z"
                coef_quad = qubo_matrix[i, j] / 4
                h_custo += SparsePauliOp("".join(pauli_str_quad), coeffs=[coef_quad])
    return h_custo

hamiltoniano_c = hamiltanino_de_custo(qubo_matrix)

In [5]:
def hamiltanino_de_mixer(valor_itens, peso_itens):
    n = len(valor_itens)
    h_mix = SparsePauliOp("I" * n, coeffs=[0])
    razao = [float(valor_itens[i]) / float(peso_itens[i]) for i in range(n)]
    max_razao = max(razao)
    normaliza = [r / max_razao for r in razao]
    for i in range(n):
        pauli_str_mix = ["I"] * n
        pauli_str_mix[i] = "X"
        h_mix += SparsePauliOp("".join(pauli_str_mix), coeffs=[normaliza[i]])
    return h_mix

hamiltoniano_m = hamiltanino_de_mixer(valor_itens, peso_itens)

In [6]:
def cria_qaoa(n, hamiltoniano_c, hamiltoniano_m, gama, beta):
    qc = QuantumCircuit(n)
    qc.h(range(n))  

    h_custo_evoluido = PauliEvolutionGate(hamiltoniano_c, time=gama)
    qc.append(h_custo_evoluido, range(n))

    h_mixer_evoluido = PauliEvolutionGate(hamiltoniano_m, time=beta)
    qc.append(h_mixer_evoluido, range(n))

    qc.measure_all()
    return qc

In [ ]:
def custo_esperado(parametros, qc, backend, shots=1000):
    atribuir_parametros_qc = qc.assign_parameters(
        {qc.parameters[0]: parametros[0], qc.parameters[1]: parametros[1]}
    )
    transpilador_qc = transpile(atribuir_parametros_qc, backend, coupling_map=None)
    result = backend.run(transpilador_qc, shots=shots).result()
    counts = result.get_counts()

    custo_total = 0
    for bitstring, frequencia in counts.items():    
        bits = [int(b) for b in bitstring[::-1]]
        custo = sum(
            qubo_matrix[i, j] * bits[i] * bits[j]
            for i in range(len(bits))
            for j in range(i, len(bits))
        )
        custo_total += custo * (frequencia / shots)

    return custo_total

In [8]:
def circuit_lr_qaoa(n, hamiltoniano_c, hamiltoniano_m, profundidade, backend, shots=1000):
    parametros_opt = []  
    qc_full = QuantumCircuit(n)
    qc_full.h(range(n))  
    
    for p in range(1, profundidade + 1):
        print(f"Otimizando camada {p}...")
        gama = ParameterVector(f"γ_{p}", 1)[0]
        beta = ParameterVector(f"β_{p}", 1)[0]
        qc_layer = cria_qaoa(n, hamiltoniano_c, hamiltoniano_m, gama, beta)
        
        
        def objective(parametros):
            return custo_esperado(parametros, qc_layer, backend, shots)
        
        # Otimizar parâmetros da camada
        initial_parametros = np.random.uniform(0, np.pi, 2)
        result = minimize(objective, initial_parametros, method="COBYLA", options={"maxiter": 100})
        parametros_opt.append(result.x)
        
        
        evo_custo = PauliEvolutionGate(hamiltoniano_c, time=result.x[0])
        evo_mixer = PauliEvolutionGate(hamiltoniano_m, time=result.x[1])
        qc_full.append(evo_custo, range(n))
        qc_full.append(evo_mixer, range(n))
    
    qc_full.measure_all()
    return qc_full, parametros_opt

In [9]:

backend = AerSimulator(method='statevector')
shots = 10000
profundidade = 4


qc, parametros_opt = circuit_lr_qaoa(qtd_itens, hamiltoniano_c, hamiltoniano_m, profundidade, backend, shots)
print("Parâmetros otimizados (γ, β):", parametros_opt)


transpilador_qc = transpile(qc, backend, coupling_map=None)
job = backend.run(transpilador_qc, shots=shots)
result = job.result()
counts = result.get_counts()
print("Contagens de bitstrings (otimizadas):")
print(counts)

# plot_histogram(counts)


Otimizando camada 1...
Otimizando camada 2...
Otimizando camada 3...
Otimizando camada 4...
Parâmetros otimizados (γ, β): [array([2.20351392, 2.17574191]), array([4.00837268, 1.56325916]), array([0.85108298, 3.91614063]), array([ 1.86729608, -0.07228913])]
Contagens de bitstrings (otimizadas):
{'00001000011110101001': 1, '00110100100000010111': 1, '10001010111110000110': 1, '10010100010101010100': 1, '11011100110110000110': 1, '11010011100000011010': 1, '01001100000101011000': 1, '01000011010101111011': 1, '01001110010100011111': 1, '11110010100110110100': 1, '10000111111110010111': 1, '11111101111101011101': 1, '00000001010011011011': 1, '01111111111010111010': 1, '01101111000011110101': 1, '01010010100101100111': 1, '00011001110001100000': 1, '11011001001001001111': 1, '01000101100011111111': 1, '10101110101110000111': 1, '00010010110111010100': 1, '10010011101000110011': 1, '10011110010011110111': 1, '11010011100011110100': 1, '01011010001110001111': 1, '00000000110000001000': 1, '0

In [10]:
def avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade):
    bits = [int(b) for b in bitstring[::-1]]
    valor_total = sum(valor_itens[i] * bits[i] for i in range(len(bits)))
    peso_total = sum(peso_itens[i] * bits[i] for i in range(len(bits)))
    valido = peso_total <= capacidade
    return valor_total, peso_total, valido

top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 soluções (otimizadas):")
melhor_valor = -float('inf')
best_bitstring = None
for bitstring, frequencia in top_results:
    valor, peso, valido = avalia_bitstring(bitstring, valor_itens, peso_itens, capacidade)
    print(f"Bitstring: {bitstring}, frequenciauência: {frequencia}, Valor: {valor}, Peso: {peso}, Válido: {valido}")
    if valido and valor > melhor_valor:
        melhor_valor = valor
        best_bitstring = bitstring

if best_bitstring is None:
    print("Nenhuma solução válida")
else:
    print(f"\nMelhor: Bitstring {best_bitstring}, Valor {melhor_valor}")

Top 10 soluções (otimizadas):
Bitstring: 01010001011100101011, frequenciauência: 3, Valor: 1618, Peso: 524, Válido: False
Bitstring: 01111011101011100010, frequenciauência: 3, Valor: 1698, Peso: 580, Válido: False
Bitstring: 11100101110001110001, frequenciauência: 3, Valor: 1591, Peso: 614, Válido: False
Bitstring: 11111011010101011100, frequenciauência: 2, Valor: 1846, Peso: 662, Válido: False
Bitstring: 00000011110010000111, frequenciauência: 2, Valor: 1320, Peso: 490, Válido: True
Bitstring: 01000011011010101111, frequenciauência: 2, Valor: 1825, Peso: 630, Válido: False
Bitstring: 00110101011100100011, frequenciauência: 2, Valor: 1551, Peso: 543, Válido: False
Bitstring: 01100010110001000001, frequenciauência: 2, Valor: 1007, Peso: 389, Válido: True
Bitstring: 11100000010101111111, frequenciauência: 2, Valor: 1974, Peso: 752, Válido: False
Bitstring: 01110010001110010000, frequenciauência: 2, Valor: 1112, Peso: 362, Válido: True

Melhor: Bitstring 00000011110010000111, Valor 1320


In [11]:
def knapsack_dp(valor, peso, capacidade):
    n = len(valor)
    dp = [[0 for _ in range(capacidade + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        for w in range(capacidade + 1):
            if peso[i-1] <= w:
                dp[i][w] = max(dp[i-1][w], dp[i-1][w - peso[i-1]] + valor[i-1])
            else:
                dp[i][w] = dp[i-1][w]
    return dp[n][capacidade]

valor_otimo = knapsack_dp(valor_itens, peso_itens, capacidade)
print(f"Valor otimo classico: {valor_otimo}")
print(f"Gap: {melhor_valor / valor_otimo:.4f}")

if melhor_valor != -float('inf'):
    print(f"A melhor solução encontrada é: {melhor_valor}")
else:
    print("( Nenhuma solução válida. )")

Valor otimo classico: 1627
Gap: 0.8113
A melhor solução encontrada é: 1320


In [13]:
# qc.draw('mpl')  
# print(qc)